In [1]:
import glob
import warnings
warnings.filterwarnings('ignore')

from osgeo import gdal
from shapely import wkt
from typing import *
%load_ext autoreload
%autoreload 2
%env PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj
%matplotlib inline

import rasterio
from torch.utils.tensorboard import SummaryWriter
import skimage
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.ops import cascaded_union
import solaris as sol

from tqdm import tqdm
import tifffile as sktif

import geopandas as gpd
import numpy as np
import cv2
from functools import partial

from fastai.imports import *
from fastai.vision import *
from fastai.metrics import dice
from fastai.callbacks import *
import ujson as json
from joblib import Parallel, delayed
import torch.nn.functional as F
import torch
import functools, traceback
def gpu_mem_restore(func):
    """Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"""
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

os.environ['FASTAI_TB_CLEAR_FRAMES']="1"
from importlib import reload

env: PROJ_LIB=/home/wwymak/anaconda3/envs/solaris/share/proj


W1118 22:09:00.263151 139684773304128 deprecation_wrapper.py:119] From /home/wwymak/anaconda3/envs/solaris2/lib/python3.7/site-packages/solaris/nets/metrics.py:103: The name tf.keras.metrics.cosine_proximity is deprecated. Please use tf.keras.losses.cosine_similarity instead.

W1118 22:09:00.333847 139684773304128 deprecation_wrapper.py:119] From /home/wwymak/anaconda3/envs/solaris2/lib/python3.7/site-packages/solaris/nets/_keras_losses.py:104: The name tf.keras.losses.cosine is deprecated. Please use tf.keras.losses.cosine_similarity instead.



In [16]:
def create_small_tiles(img_filepath, save_dir_rgb):
    img_rgb = PIL.Image.open(img_filepath)
    img_id = img_filepath.name.replace('.png', '')

    boxes = [
        (0, 0, 512, 512),
        (0, 256, 512, 768),
        (0, 512, 512, 1024),
        (256, 0, 768, 512),
        (256, 256, 768, 768),
        (256, 512, 768, 1024),
        (512, 0, 1024, 512),
        (512, 256, 1024, 768),
        (512, 512, 1024, 1024)
    ]
    im_crops = [img_rgb.crop(box) for box in boxes]

    for i, im_crop in enumerate(im_crops):
        im_crop.save(save_dir_rgb / f"rgb_{img_id}_{i}.png")

In [9]:
data_dir = Path("/media/wwymak/Storage/xView2")
mask_dir = data_dir / "mask_full_size_single_channel"


test_images_crops = data_dir / "test_crops"
test_mask_crops = data_dir / "test_mask_crops_single_channel"
test_images_crops.mkdir(exist_ok=True)
test_mask_crops.mkdir(exist_ok=True)

undamaged_fnames = [fname for fname in (data_dir / "test" / "images").ls() if '_pre_' in fname.name]

undamaged_fnames[:5]

[PosixPath('/media/wwymak/Storage/xView2/test/images/test_pre_00909.png'),
 PosixPath('/media/wwymak/Storage/xView2/test/images/test_pre_00371.png'),
 PosixPath('/media/wwymak/Storage/xView2/test/images/test_pre_00824.png'),
 PosixPath('/media/wwymak/Storage/xView2/test/images/test_pre_00270.png'),
 PosixPath('/media/wwymak/Storage/xView2/test/images/test_pre_00186.png')]

In [17]:
img_ids = [f.name.replace('.png', '') for f in undamaged_fnames]

create_tile = partial(create_small_tiles, save_dir_rgb=test_images_crops)

# [create_tile(img_filepath, mask_filepath, im_id) for (img_filepath, mask_filepath, im_id) in zip(img_filepaths, mask_filepaths, img_ids )]
_ = Parallel(n_jobs=14)(delayed(create_tile)(img_filepath) \
                       for img_filepath in undamaged_fnames)
# fro